In [ ]:
# ercot_volatility_analysis.py (Plotly version, cleaned + EGARCH with t-dist + trimming + prep for GARCH-X)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt  # Only for ACF/PACF as Plotly doesn't support these directly

# === 1. Load combined dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=['datetime'])

# === 2. Select zone and calculate log returns ===
zone = 'LZ_HOUSTON'
price_col = f'{zone}_rtm'
df = df[['datetime', price_col, 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[price_col] > 0]  # Filter out non-positive prices

# Calculate log returns
df['log_return'] = np.log(df[price_col]).diff() * 100

# Drop rows with NaN or inf in log_return
df['log_return'] = df['log_return'].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['log_return'])

# === Trim extreme outliers in log return ===
outlier_threshold = 300
df['log_return'] = df['log_return'].clip(lower=-outlier_threshold, upper=outlier_threshold)

# === 3. Plot distribution of trimmed returns ===
fig = px.histogram(df, x='log_return', nbins=100, marginal='rug',
                   title=f'Trimmed Log Return Distribution – {zone}',
                   labels={'log_return': 'Log Return (%)'})
fig.update_layout(template='plotly_white')
fig.show()

# === 4. ADF Test ===
adf_result = adfuller(df['log_return'])
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")

# === 5. ACF/PACF Plots (Matplotlib) ===
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(df['log_return'], lags=40, ax=ax[0])
plot_pacf(df['log_return'], lags=40, ax=ax[1])
ax[0].set_title('ACF of Log Returns')
ax[1].set_title('PACF of Log Returns')
plt.tight_layout()
plt.show()

# === 6. Fit GARCH(1,1) model ===
log_returns_clean = df['log_return'].dropna()
model_garch = arch_model(log_returns_clean, vol='Garch', p=1, q=1)
res_garch = model_garch.fit(disp='off')
df = df.loc[log_returns_clean.index].copy()
df['volatility_garch'] = res_garch.conditional_volatility.values

print("\nGARCH(1,1) Parameters:")
print(res_garch.summary())

# === 7. Fit EGARCH(1,1) with t-distribution ===
model_egarch = arch_model(log_returns_clean, vol='EGarch', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')
df['volatility_egarch'] = res_egarch.conditional_volatility.values

print("\nEGARCH(1,1) Parameters (t-distribution):")
print(res_egarch.summary())

# === 8. Plot both volatilities ===
fig = px.line(df, x='datetime', y=['volatility_garch', 'volatility_egarch'],
              title=f'GARCH vs EGARCH (t-dist) Conditional Volatility – {zone}',
              labels={'value': 'Volatility (%)', 'datetime': 'Date', 'variable': 'Model'})
fig.update_layout(template='plotly_white')
fig.show()

# === 9. Segment and compare volatility (before/after 2024) ===
df['period'] = np.where(df['datetime'] < '2024-01-01', 'pre_BESS', 'post_BESS')
mean_vols = df.groupby('period')[['volatility_garch', 'volatility_egarch']].mean()
print("\nAverage Volatility by Period:")
print(mean_vols)

# === 10. Scatter matrix: volatility vs renewables/BESS ===
fig = px.scatter_matrix(df.dropna(subset=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw']),
                        dimensions=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw'],
                        title='EGARCH Volatility vs Renewables and BESS')
fig.update_layout(template='plotly_white', height=700)
fig.show()

# === Ready for future: GARCH-X (with exogenous variables) ===
# Example:
# model_garchx = arch_model(log_returns_clean, vol='Garch', p=1, q=1, x=df[['wind_mw', 'solar_mw', 'net_output_mw']].shift(1))


In [ ]:
# ercot_volatility_analysis.py

import pandas as pd
import numpy as np
import plotly.express as px
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# === 1. Load data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=['datetime'])

# === 2. Preprocessing ===
zone = 'LZ_HOUSTON'
price_col = f'{zone}_rtm'
df = df[['datetime', price_col, 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[price_col] > 0]
df['log_return'] = np.log(df[price_col]).diff() * 100
df['log_return'] = df['log_return'].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['log_return'])
df['log_return'] = df['log_return'].clip(-300, 300)

# === 3. ADF Test ===
adf_stat, p_value, *_ = adfuller(df['log_return'])
print(f"ADF Statistic: {adf_stat:.4f}, p-value: {p_value:.4f}")

# === 4. ACF/PACF ===
fig, ax = plt.subplots(2, 1, figsize=(12, 6))
plot_acf(df['log_return'], lags=40, ax=ax[0])
plot_pacf(df['log_return'], lags=40, ax=ax[1])
ax[0].set_title('ACF of Log Returns')
ax[1].set_title('PACF of Log Returns')
plt.tight_layout()
plt.show()

# === 5. EGARCH Fit ===
log_returns_clean = df['log_return']
model_egarch = arch_model(log_returns_clean, vol='EGarch', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')
df['volatility_egarch'] = res_egarch.conditional_volatility.values
print(res_egarch.summary())

# === 6. Time series plot ===
fig = px.line(df, x='datetime', y='volatility_egarch', title='EGARCH Volatility Over Time')
fig.update_layout(template='plotly_white')
fig.show()

# === 7. Boxplot Pre vs Post BESS ===
df['period'] = np.where(df['datetime'] < '2024-01-01', 'Pre-BESS', 'Post-BESS')
fig = px.box(df, x='period', y='volatility_egarch', title='EGARCH Volatility Pre vs Post BESS')
fig.update_layout(template='plotly_white')
fig.show()

# === 8. Scatter Matrix ===
fig = px.scatter_matrix(df.dropna(subset=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw']),
                        dimensions=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw'],
                        title='EGARCH Volatility vs Renewables and BESS')
fig.update_layout(template='plotly_white', height=700)
fig.show()


In [ ]:
import pandas as pd
import numpy as np
from arch import arch_model
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === Load data ===
df = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv", parse_dates=['datetime'])

# === Define zones to analyze ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Results containers ===
summary_list = []
volatility_df = pd.DataFrame({'datetime': df['datetime']})

# === Loop over zones ===
for zone in zones:
    price_col = f'{zone}_rtm'
    if price_col not in df.columns:
        print(f"❌ Missing column: {price_col}, skipping...")
        continue

    temp = df[['datetime', price_col]].copy()
    temp = temp[temp[price_col] > 0].dropna()
    temp['log_return'] = np.log(temp[price_col]).diff() * 100
    temp['log_return'] = temp['log_return'].replace([np.inf, -np.inf], np.nan).clip(-300, 300)
    temp = temp.dropna(subset=['log_return'])

    # === Fit EGARCH(1,1) with t-distribution ===
    model = arch_model(temp['log_return'], vol='EGarch', p=1, q=1, dist='t')
    res = model.fit(disp='off')

    vol_col = f"{zone}_vol_egarch"
    volatility_df = volatility_df.merge(temp[['datetime']].assign(**{vol_col: res.conditional_volatility.values}), on='datetime', how='left')

    # === Collect summary ===
    summary_list.append({
        'Zone': zone,
        'LogLik': round(res.loglikelihood, 2),
        'AIC': round(res.aic, 1),
        'ν (t-dist)': round(res.params.get('nu', np.nan), 2),
        'ω': round(res.params.get('omega', np.nan), 4),
        'α1': round(res.params.get('alpha[1]', np.nan), 4),
        'β1': round(res.params.get('beta[1]', np.nan), 4),
        'mu': round(res.params.get('mu', np.nan), 4)
    })

# === Create summary table ===
summary_df = pd.DataFrame(summary_list)
print("\n📊 EGARCH Model Summary Per Zone:")
print(summary_df.to_string(index=False))

# === Plot 4 volatility curves as subplots ===
fig = make_subplots(rows=2, cols=2, shared_xaxes=True, subplot_titles=zones)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for i, zone in enumerate(zones):
    vol_col = f"{zone}_vol_egarch"
    if vol_col not in volatility_df.columns:
        continue
    row, col = positions[i]
    fig.add_trace(go.Scatter(x=volatility_df['datetime'], y=volatility_df[vol_col],
                             mode='lines', name=zone,
                             line=dict(width=1)), row=row, col=col)

fig.update_layout(title='EGARCH Conditional Volatility by Zone',
                  template='plotly_white', height=700, showlegend=False)
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Volatility (%)')
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define cutoff
cutoff = pd.to_datetime("2024-01-01")

# Smoothing: 1-month rolling average (hourly data = 24*30 = 720)
window_hours = 720
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
vol_df = volatility_df.copy()

# Create smoothed series and flag pre/post-BESS
for zone in zones:
    col = f"{zone}_vol_egarch"
    vol_df[f"{col}_smooth"] = vol_df[col].rolling(window=window_hours, min_periods=1).mean()
vol_df['period'] = np.where(vol_df['datetime'] < cutoff, 'Pre-BESS', 'Post-BESS')

# Plotting
fig = make_subplots(rows=2, cols=2, subplot_titles=zones, shared_xaxes=True, shared_yaxes=True)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for i, zone in enumerate(zones):
    row, col = positions[i]
    col_smooth = f"{zone}_vol_egarch_smooth"
    
    pre = vol_df[vol_df['datetime'] < cutoff]
    post = vol_df[vol_df['datetime'] >= cutoff]
    
    # Add pre-BESS smoothed line
    fig.add_trace(go.Scatter(
        x=pre['datetime'], y=pre[col_smooth],
        mode='lines', name=f"{zone} Pre-BESS",
        line=dict(color='gray'), showlegend=False
    ), row=row, col=col)

    # Add post-BESS smoothed line
    fig.add_trace(go.Scatter(
        x=post['datetime'], y=post[col_smooth],
        mode='lines', name=f"{zone} Post-BESS",
        line=dict(color='steelblue'), showlegend=False
    ), row=row, col=col)

fig.update_layout(
    height=700, width=1100,
    title_text="Rolling EGARCH Volatility by Zone (Pre/Post BESS)",
    template="plotly_white"
)
fig.update_yaxes(title_text="Volatility (%)")
fig.update_xaxes(title_text="Date")
fig.update_xaxes(showticklabels=True, tickformat="%Y")  # Show year labels
fig.update_layout(xaxis_showticklabels=True)
fig.show()


In [ ]:
# ercot_volatility_analysis.py (Enhanced with diagnostics)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from arch import arch_model
from matplotlib import cm
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones and colormap ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
colorscale = cm.get_cmap('Blues', 4)

zone_colors = {
    zone: 'rgba({},{},{},{})'.format(*((np.array(colorscale(i)[:3]) * 255).astype(int)), 0.6)
    for i, zone in enumerate(zones)
}

bess_start_date = pd.to_datetime('2024-01-01')
fig_bess = make_subplots(
    rows=2, cols=2, subplot_titles=zones,
    vertical_spacing=0.12, horizontal_spacing=0.08
)

summary_table = []

for i, zone in enumerate(zones):
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col]].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan).clip(-3, 3)
    df = df.dropna(subset=['log_return'])

    df_pre = df[df['datetime'] < bess_start_date].copy()
    df_post = df[df['datetime'] >= bess_start_date].copy()

    try:
        res_pre = arch_model(df_pre['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_pre['vol'] = res_pre.conditional_volatility
    except:
        df_pre['vol'] = np.nan

    try:
        res_post = arch_model(df_post['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_post['vol'] = res_post.conditional_volatility
    except:
        df_post['vol'] = np.nan

    df_pre['year'] = df_pre['datetime'].dt.year
    df_post['year'] = df_post['datetime'].dt.year
    avg_yearly = pd.concat([df_pre, df_post]).groupby('year')['vol'].mean().reset_index()

    # Rolling average
    df_pre['rolling_vol'] = df_pre['vol'].rolling(24 * 30).mean()
    df_post['rolling_vol'] = df_post['vol'].rolling(24 * 30).mean()

    r, c = i // 2 + 1, i % 2 + 1
    color = zone_colors[zone]

    fig_bess.add_trace(go.Scatter(
        x=df_pre['datetime'], y=df_pre['rolling_vol'], mode='lines',
        name=f'{zone} Pre-BESS', line=dict(color='lightgray', width=1.2),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=df_post['datetime'], y=df_post['rolling_vol'], mode='lines',
        name=f'{zone} Post-BESS', line=dict(color=color, width=1.5),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=avg_yearly['year'], y=avg_yearly['vol'],
        mode='lines+text', line=dict(color='black', dash='dash'),
        text=avg_yearly['vol'].round(1), textposition='top right',
        name=f'{zone} Yearly Avg', showlegend=False
    ), row=r, col=c)

    # Summary stats and statistical tests
    mean_pre = df_pre['vol'].mean()
    mean_post = df_post['vol'].mean()
    std_pre = df_pre['vol'].std()
    std_post = df_post['vol'].std()
    _, p_ttest = ttest_ind(df_pre['vol'].dropna(), df_post['vol'].dropna(), equal_var=False)
    _, p_mw = mannwhitneyu(df_pre['vol'].dropna(), df_post['vol'].dropna(), alternative='two-sided')

    # Residual diagnostics (only on post)
    lb_pval = het_arch(res_post.resid.dropna())[1] if res_post else np.nan
    arch_pval = acorr_ljungbox(res_post.resid.dropna(), lags=[12], return_df=True)['lb_pvalue'].iloc[0] if res_post else np.nan

    summary_table.append([
        zone, round(mean_pre, 2), round(mean_post, 2),
        round(std_pre, 1), round(std_post, 1),
        round(p_ttest, 4), round(p_mw, 4), round(lb_pval, 4), round(arch_pval, 4)
    ])

# Set subplot font size and layout
for ann in fig_bess['layout']['annotations']:
    ann['font'] = dict(size=14)

fig_bess.update_layout(
    height=950,
    title="Rolling GARCH Volatility – Pre vs Post BESS",
    plot_bgcolor='white',
    font=dict(family="Arial", size=12),
    margin=dict(l=40, r=40, t=60, b=40)
)
fig_bess.show()

# Print statistical summary
sum_df = pd.DataFrame(summary_table, columns=[
    "Zone", "Pre-BESS Mean", "Post-BESS Mean", "Pre Std", "Post Std",
    "p-value (t-test)", "p-value (MWU)", "ARCH Test (p)", "Ljung-Box (p)"
])
print("\nStatistical Summary of Volatility Pre vs Post BESS:")
print(sum_df.to_string(index=False))


In [ ]:
# ercot_volatility_analysis.py (Enhanced with diagnostics and GARCH-X)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from arch import arch_model
from matplotlib import cm
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones and colormap ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
colorscale = cm.get_cmap('Blues', 4)

zone_colors = {
    'LZ_HOUSTON': 'rgba(40, 80, 180, 1.0)',   # custom dark blue
    'LZ_NORTH': 'rgba(80, 120, 200, 1.0)',
    'LZ_SOUTH': 'rgba(120, 160, 220, 1.0)',
    'LZ_WEST': 'rgba(160, 200, 240, 1.0)'
}

bess_start_date = pd.to_datetime('2024-01-01')
fig_bess = make_subplots(
    rows=2, cols=2, subplot_titles=zones,
    vertical_spacing=0.12, horizontal_spacing=0.08
)

summary_table = []

for i, zone in enumerate(zones):
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'wind_mw', 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan).clip(-3, 3)
    df = df.dropna(subset=['log_return'])

    df['wind_std'] = df['wind_mw'].rolling(24).std().fillna(method='bfill')
    df['solar_std'] = df['solar_mw'].rolling(24).std().fillna(method='bfill')

    df_pre = df[df['datetime'] < bess_start_date].copy()
    df_post = df[df['datetime'] >= bess_start_date].copy()

    try:
        res_pre = arch_model(df_pre['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_pre['vol'] = res_pre.conditional_volatility
    except:
        df_pre['vol'] = np.nan

    try:
        res_post = arch_model(df_post['log_return'] * 100, vol='Garch', p=1, q=1).fit(disp='off')
        df_post['vol'] = res_post.conditional_volatility
    except:
        df_post['vol'] = np.nan

    df_pre['year'] = df_pre['datetime'].dt.year
    df_post['year'] = df_post['datetime'].dt.year
    avg_yearly = pd.concat([df_pre, df_post]).groupby('year')['vol'].mean().reset_index()

    # Rolling average
    df_pre['rolling_vol'] = df_pre['vol'].rolling(24 * 30).mean()
    df_post['rolling_vol'] = df_post['vol'].rolling(24 * 30).mean()

    r, c = i // 2 + 1, i % 2 + 1
    color = zone_colors[zone]

    fig_bess.add_trace(go.Scatter(
        x=df_pre['datetime'], y=df_pre['rolling_vol'], mode='lines',
        name=f'{zone} Pre-BESS', line=dict(color='lightgray', width=1.2),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=df_post['datetime'], y=df_post['rolling_vol'], mode='lines',
        name=f'{zone} Post-BESS', line=dict(color=color, width=2.2),
        showlegend=(i == 0)
    ), row=r, col=c)

    fig_bess.add_trace(go.Scatter(
        x=avg_yearly['year'], y=avg_yearly['vol'],
        mode='lines+text', line=dict(color='black', dash='dash'),
        text=avg_yearly['vol'].round(1), textposition='top right',
        name=f'{zone} Yearly Avg', showlegend=False
    ), row=r, col=c)

    # Summary stats and statistical tests
    mean_pre = df_pre['vol'].mean()
    mean_post = df_post['vol'].mean()
    std_pre = df_pre['vol'].std()
    std_post = df_post['vol'].std()
    _, p_ttest = ttest_ind(df_pre['vol'].dropna(), df_post['vol'].dropna(), equal_var=False)
    _, p_mw = mannwhitneyu(df_pre['vol'].dropna(), df_post['vol'].dropna(), alternative='two-sided')

    # Residual diagnostics (only on post)
    lb_pval = het_arch(res_post.resid.dropna())[1] if res_post else np.nan
    arch_pval = acorr_ljungbox(res_post.resid.dropna(), lags=[12], return_df=True)['lb_pvalue'].iloc[0] if res_post else np.nan

    summary_table.append([
        zone, round(mean_pre, 2), round(mean_post, 2),
        round(std_pre, 1), round(std_post, 1),
        round(p_ttest, 4), round(p_mw, 4), round(lb_pval, 4), round(arch_pval, 4)
    ])

# Set subplot font size and layout
for ann in fig_bess['layout']['annotations']:
    ann['font'] = dict(size=14)

fig_bess.update_layout(
    height=950,
    title="Rolling GARCH Volatility – Pre vs Post BESS",
    plot_bgcolor='white',
    font=dict(family="Arial", size=12),
    margin=dict(l=40, r=40, t=60, b=40)
)
fig_bess.show()

# Plot Wind and Solar Daily Std Dev with Different Colors
fig_ws = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=["Wind MW Daily Std Dev", "Solar MW Daily Std Dev"])
fig_ws.add_trace(go.Scatter(
    x=full_df['datetime'], y=full_df['wind_mw'].rolling(24).std(),
    line=dict(color='teal'), name="Wind Std Dev"
), row=1, col=1)
fig_ws.add_trace(go.Scatter(
    x=full_df['datetime'], y=full_df['solar_mw'].rolling(24).std(),
    line=dict(color='orange'), name="Solar Std Dev"
), row=2, col=1)
fig_ws.update_layout(title="Daily Std Dev of Wind and Solar (MW)", height=700)
fig_ws.show()

# Print statistical summary
sum_df = pd.DataFrame(summary_table, columns=[
    "Zone", "Pre-BESS Mean", "Post-BESS Mean", "Pre Std", "Post Std",
    "p-value (t-test)", "p-value (MWU)", "ARCH Test (p)", "Ljung-Box (p)"
])
sum_df.to_csv("volatility_summary_stats.csv", index=False)
print("\nStatistical Summary of Volatility Pre vs Post BESS:")
print(sum_df.to_string(index=False))


In [ ]:
# ercot_volatility_analysis.py (Post-BESS GARCH-X Only)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from arch import arch_model

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones and colormap ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
zone_colors = {
    'LZ_HOUSTON': 'rgba(20, 40, 90, 1.0)',
    'LZ_NORTH': 'rgba(40, 60, 120, 1.0)',
    'LZ_SOUTH': 'rgba(60, 90, 150, 1.0)',
    'LZ_WEST': 'rgba(80, 120, 180, 1.0)'
}

bess_start_date = pd.to_datetime('2024-01-01')
fig_gx = make_subplots(
    rows=2, cols=2, subplot_titles=zones,
    vertical_spacing=0.12, horizontal_spacing=0.08
)

def safe_post_garchx(data, exog_cols):
    X = data[exog_cols].copy()
    low_var_cols = [col for col in exog_cols if X[col].std() < 1e-3 or X[col].isna().all()]
    X = X.drop(columns=low_var_cols)
    if X.shape[1] == 0:
        return None, np.nan, np.nan, np.nan
    try:
        model = arch_model(data['log_return'] * 100, vol='Garch', p=1, q=1, x=X)
        result = model.fit(disp='off')
        coefs = [result.params.get(f'x{i+1}', np.nan) for i in range(len(X.columns))]
        return result, *coefs + [np.nan] * (3 - len(coefs))
    except:
        return None, np.nan, np.nan, np.nan

coeff_table = []

for i, zone in enumerate(zones):
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'wind_mw', 'solar_mw', 'charging_mw', 'discharging_mw', 'net_output_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan).clip(-3, 3)
    df = df.dropna(subset=['log_return'])

    df['wind_std'] = df['wind_mw'].rolling(24).std().fillna(method='bfill')
    df['solar_std'] = df['solar_mw'].rolling(24).std().fillna(method='bfill')
    df['bess_std'] = df['net_output_mw'].rolling(24).std().fillna(method='bfill')

    df_post = df[df['datetime'] >= bess_start_date].copy()
    res_post, wind_post, solar_post, bess_post = safe_post_garchx(df_post, ['wind_std', 'solar_std', 'bess_std'])

    if res_post:
        df_post['vol'] = res_post.conditional_volatility
    else:
        df_post['vol'] = np.nan

    coeff_table.append({
        'Zone': zone, 'Period': 'Post-BESS',
        'Wind Coef': wind_post, 'Solar Coef': solar_post, 'BESS Coef': bess_post
    })

    df_post['rolling_vol'] = df_post['vol'].rolling(24 * 30).mean()
    r, c = i // 2 + 1, i % 2 + 1
    color = zone_colors[zone]

    fig_gx.add_trace(go.Scatter(
        x=df_post['datetime'], y=df_post['rolling_vol'], mode='lines',
        name=f'{zone} Post-BESS', line=dict(color=color, width=2.5),
        showlegend=(i == 0)
    ), row=r, col=c)

fig_gx.update_layout(
    height=900,
    title="GARCH-X Volatility (Post-BESS only): Wind, Solar, BESS as Exogenous Variables",
    plot_bgcolor='white',
    font=dict(family="Arial", size=12),
    margin=dict(l=40, r=40, t=60, b=40)
)
fig_gx.show()

# === Coefficient Table ===
coef_df = pd.DataFrame(coeff_table)
print("\n=== GARCH-X Coefficients (Post-BESS Only) ===")
print(coef_df.round(4).to_string(index=False))


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Focus on one zone (e.g. LZ_HOUSTON) ===
zone = 'LZ_HOUSTON'
df = full_df[['datetime', f'{zone}_rtm', 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[f'{zone}_rtm'] > 0]

# === Create volatility proxies (rolling std dev) ===
df['wind_std'] = df['wind_mw'].rolling(24).std().bfill()
df['solar_std'] = df['solar_mw'].rolling(24).std().bfill()
df['bess_std'] = df['net_output_mw'].rolling(24).std().bfill()

# === Filter post-BESS ===
df = df[df['datetime'] >= '2024-01-01'].copy()
df.dropna(subset=['wind_std', 'solar_std', 'bess_std'], inplace=True)

# === Correlation matrix ===
print("\n🔗 Correlation Matrix:")
corr = df[['wind_std', 'solar_std', 'bess_std']].corr()
print(corr.round(3))

# === VIF calculation ===
X = df[['wind_std', 'solar_std', 'bess_std']]
X = (X - X.mean()) / X.std()  # Standardize
vif_data = pd.DataFrame()
vif_data['Regressor'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n📊 Variance Inflation Factor (VIF):")
print(vif_data)

# === Optional heatmap ===
plt.figure(figsize=(5, 4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap – Volatility Regressors")
plt.tight_layout()
plt.show()


In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Solar Only, Full Period + Summary Visuals)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Prepare output ===
results = []
summary_labels = []

for zone in zones:
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
    df['solar_std'] = df['solar_mw'].rolling(72).std().bfill()
    df = df.dropna(subset=['log_return', 'solar_std'])

    y = df['log_return'] * 100
    x = df[['solar_std']]
    x = (x - x.mean()) / x.std()

    try:
        model = arch_model(y, vol='Garch', p=1, q=1, x=x)
        res = model.fit(disp='off')
        coef = res.params.get('x[0]', np.nan)
        pval = res.pvalues.get('x[0]', np.nan)
        sig_label = (
            "✅ Strong effect" if pval < 0.01 else
            "🔸 Weak effect" if pval < 0.1 else
            "❌ Not significant"
        )
        summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

        print(f"\n[{zone} – solar_std]\n")
        print(res.summary())

    except Exception as e:
        coef = np.nan
        pval = np.nan
        summary_labels.append(f"{zone}: model failed")
        print(f"Model failed for {zone}: {e}")

    results.append({
        'Zone': zone,
        'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
        'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
    })

# === Display Results ===
result_df = pd.DataFrame(results)
print("\n=== GARCH-X Solar Coefficients (Full Period) ===")
print(result_df.set_index('Zone'))

# === Plot: Summary Bar Chart with Annotations ===
fig = go.Figure()
fig.add_trace(go.Bar(
    x=result_df['Zone'],
    y=result_df['Coef'],
    marker_color='darkorange',
    text=summary_labels,
    textposition="outside"
))
fig.update_layout(
    title="GARCH-X Coefficients for Solar – Full Period",
    xaxis_title="Zone",
    yaxis_title="Coefficient Value",
    height=500,
    margin=dict(l=60, r=60, t=60, b=60)
)
fig.show()


In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Solar Only, Full Period + Summary Visuals + Volatility Trend Plots)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Prepare output ===
results = []
summary_labels = []
vol_trends = {}

for zone in zones:
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
    df['solar_std'] = df['solar_mw'].rolling(72).std().bfill()
    df = df.dropna(subset=['log_return', 'solar_std'])

    y = df['log_return'] * 100
    x = df[['solar_std']]
    x = (x - x.mean()) / x.std()

    try:
        model = arch_model(y, vol='Garch', p=1, q=1, x=x)
        res = model.fit(disp='off')
        coef = res.params.get('x[0]', np.nan)
        pval = res.pvalues.get('x[0]', np.nan)
        cond_vol = res.conditional_volatility

        df['cond_vol'] = cond_vol
        vol_trends[zone] = df[['datetime', 'cond_vol', 'solar_std']].copy()

        sig_label = (
            "✅ Strong effect" if pval < 0.01 else
            "🔸 Weak effect" if pval < 0.1 else
            "❌ Not significant"
        )
        summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

        print(f"\n[{zone} – solar_std]\n")
        print(res.summary())

    except Exception as e:
        coef = np.nan
        pval = np.nan
        summary_labels.append(f"{zone}: model failed")
        print(f"Model failed for {zone}: {e}")

    results.append({
        'Zone': zone,
        'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
        'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
    })

# === Display Results ===
result_df = pd.DataFrame(results)
print("\n=== GARCH-X Solar Coefficients (Full Period) ===")
print(result_df.set_index('Zone'))

# === Plot: Summary Bar Chart with Annotations ===
fig = go.Figure()
fig.add_trace(go.Bar(
    x=result_df['Zone'],
    y=result_df['Coef'],
    marker_color='darkorange',
    text=summary_labels,
    textposition="outside"
))
fig.update_layout(
    title="GARCH-X Coefficients for Solar – Full Period",
    xaxis_title="Zone",
    yaxis_title="Coefficient Value",
    height=500,
    margin=dict(l=60, r=60, t=60, b=60)
)
fig.show()

# === Plot: Time Series of Volatility vs. Solar_STD ===
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8), sharex=True)
axes = axes.flatten()

for i, zone in enumerate(zones):
    ax = axes[i]
    if zone in vol_trends:
        zdf = vol_trends[zone]
        ax.plot(zdf['datetime'], zdf['cond_vol'], label='Volatility (GARCH)', color='navy')
        ax.plot(zdf['datetime'], zdf['solar_std'], label='Solar Std Dev', color='orange', alpha=0.6)
        ax.set_title(zone)
        ax.set_ylabel("Scaled Values")
        ax.legend(loc='upper right')

fig.suptitle("Solar Variability vs. Conditional Volatility – GARCH-X Trend", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Wind + BESS, Full Period)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Loop over regressors: wind and bess ===
for regressor in ['wind_mw', 'net_output_mw']:
    reg_label = 'Wind' if regressor == 'wind_mw' else 'BESS'
    reg_std = regressor.replace('_mw', '_std')

    results = []
    summary_labels = []
    vol_trends = {}

    for zone in zones:
        price_col = f'{zone}_rtm'
        df = full_df[['datetime', price_col, regressor]].copy()
        df = df[df[price_col] > 0]
        df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
        df[reg_std] = df[regressor].rolling(72).std().bfill()
        df = df.dropna(subset=['log_return', reg_std])

        y = df['log_return'] * 100
        x = df[[reg_std]]
        x = (x - x.mean()) / x.std()

        try:
            model = arch_model(y, vol='Garch', p=1, q=1, x=x)
            res = model.fit(disp='off')
            coef = res.params.get('x[0]', np.nan)
            pval = res.pvalues.get('x[0]', np.nan)
            cond_vol = res.conditional_volatility

            df['cond_vol'] = cond_vol
            vol_trends[zone] = df[['datetime', 'cond_vol', reg_std]].copy()

            sig_label = (
                "✅ Strong effect" if pval < 0.01 else
                "🔸 Weak effect" if pval < 0.1 else
                "❌ Not significant"
            )
            summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

            print(f"\n[{zone} – {reg_std}]")
            print(res.summary())

        except Exception as e:
            coef = np.nan
            pval = np.nan
            summary_labels.append(f"{zone}: model failed")
            print(f"Model failed for {zone}: {e}")

        results.append({
            'Zone': zone,
            'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
            'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
        })

    # === Plot Coefficient Summary ===
    result_df = pd.DataFrame(results)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=result_df['Zone'],
        y=result_df['Coef'],
        marker_color='teal' if regressor == 'wind_mw' else 'crimson',
        text=summary_labels,
        textposition="outside"
    ))
    fig.update_layout(
        title=f"GARCH-X Coefficients for {reg_label} – Full Period",
        xaxis_title="Zone",
        yaxis_title="Coefficient Value",
        height=500,
        margin=dict(l=60, r=60, t=60, b=60)
    )
    fig.show()

    # === Plot Time Series Trends ===
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8), sharex=True)
    axes = axes.flatten()

    for i, zone in enumerate(zones):
        ax = axes[i]
        if zone in vol_trends:
            zdf = vol_trends[zone]
            ax.plot(zdf['datetime'], zdf['cond_vol'], label='Volatility (GARCH)', color='navy')
            ax.plot(zdf['datetime'], zdf[reg_std], label=f'{reg_label} Std Dev', alpha=0.6)
            ax.set_title(zone)
            ax.set_ylabel("Scaled Values")
            ax.legend(loc='upper right')

    fig.suptitle(f"{reg_label} Variability vs. Conditional Volatility – GARCH-X Trend", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

# === Load and preprocess ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=["datetime"])

zone = "LZ_HOUSTON"
price_col = f"{zone}_rtm"

# Filter and clean
df = df[df[price_col] > 0].copy()
df["log_return"] = np.log(df[price_col]).diff().clip(-3, 3) * 100
df["wind_std"] = df["wind_mw"].rolling(24).std().bfill()
df = df.dropna(subset=["log_return", "wind_std"])

y = df["log_return"]
x = (df[["wind_std"]] - df[["wind_std"]].mean()) / df[["wind_std"]].std()

# === GARCH-X ===
print("Fitting GARCH-X...")
model_garchx = arch_model(y, vol="Garch", p=1, q=1, x=x)
res_garchx = model_garchx.fit(disp="off")
print(res_garchx.summary())

# === TGARCH (GJR-GARCH) ===
print("\nFitting TGARCH...")
model_tgarch = arch_model(y, vol="GARCH", p=1, o=1, q=1)
res_tgarch = model_tgarch.fit(disp="off")
print(res_tgarch.summary())

# === Regime Switching ===
print("\nFitting Regime-Switching...")
model_rs = MarkovRegression(y, k_regimes=2, trend="c", switching_variance=True)
res_rs = model_rs.fit()
print(res_rs.summary())

# === Model Comparison ===
print("\n=== AIC / BIC Comparison ===")
aic_bic_df = pd.DataFrame({
    "Model": ["GARCH-X", "TGARCH", "Regime Switching"],
    "AIC": [res_garchx.aic, res_tgarch.aic, res_rs.aic],
    "BIC": [res_garchx.bic, res_tgarch.bic, res_rs.bic]
})
print(aic_bic_df)

# === Plot: Regime Probabilities ===
res_rs.smoothed_marginal_probabilities[0].plot(figsize=(12, 4), title="Probability of Regime 0 (Low Volatility)")
plt.xlabel("Time")
plt.ylabel("Probability")
plt.grid(True)
plt.tight_layout()
plt.show()

# === Plot: Conditional Volatility ===
plt.figure(figsize=(12, 4))
plt.plot(res_garchx.conditional_volatility, label="GARCH-X", alpha=0.8)
plt.plot(res_tgarch.conditional_volatility, label="TGARCH", alpha=0.8)
plt.title("Conditional Volatility Comparison")
plt.ylabel("Volatility")
plt.xlabel("Time")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()
